<a href="https://colab.research.google.com/github/nananair/Research-NLP-projects/blob/main/Regression_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides a researcher-friendly template for analysing skewed count data (e.g., social media shares, likes, comments, applause counts, laughter counts) with automatic model selection, diagnostics, and clear interpretation guidelines.

FEATURES:
- Automatic model selection (Poisson → Negative Binomial → Zero-Inflated NB)
- Comprehensive diagnostics (dispersion, zero-inflation, multicollinearity)
- Visual diagnostic plots
- Export-ready results tables

ADAPTABLE TO:
- Social media engagement (shares, likes, retweets)
- Parliamentary discourse (applause, interruptions, laughter)
- Media coverage (headlines, mentions)
- Academic impact (citations, downloads)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from google.colab import files

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    PLOT_AVAILABLE = True
except ImportError:
    PLOT_AVAILABLE = False
    print("Matplotlib/Seaborn not available. Skipping diagnostic plots.")


In [2]:
print("Please upload your dataset (Excel or CSV format)")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(".xlsx"):
        df = pd.read_excel(filename)
    elif filename.endswith(".csv"):
        df = pd.read_csv(filename)
    else:
        raise ValueError("File must be .xlsx or .csv")
    break

print(f"Dataset loaded: {len(df)} rows, {len(df.columns)} columns")
print("\nFirst 5 rows:")
print(df.head())

df.columns = df.columns.str.strip().str.replace(r"\s+", "_", regex=True)

Please upload your dataset (Excel or CSV format)


Saving updated_thesis_dataset_v2.xlsx to updated_thesis_dataset_v2.xlsx
Dataset loaded: 192 rows, 16 columns

First 5 rows:
                                            Articles  Outlet_type  \
0  Second police officer died by suicide followin...  Online-only   
1  Rioters breached US Capitol security on Wednes...      Popular   
2  ‘They Got a Officer!’: How a Mob Dragged and B...      Quality   
3  As the D.C. police clear the Capitol grounds, ...      Quality   
4  Now it’s sinking in: Wednesday’s Capitol Hill ...      Popular   

   Word_length  Share_counts_Twitter  Share_counts_Facebook Dataset  \
0        247.0                 21800                  21200     USA   
1        622.0                 19800                  24100     USA   
2        441.0                 12900                  23400     USA   
3        468.0                 16400                  20100     USA   
4       1042.0                 17200                  23700     USA   

   Eliteness  Personalisation  Sup

In [3]:
CONFIG = {
    # ANALYSIS MODE
    "MODE": "AUTO",  # Options: "SIMPLE" (basic Negative Binomial only), "AUTO" (automatic selection), "ADVANCED" (include ZINB)

    # COLUMN SPECIFICATIONS
    "CASE_COL": "Dataset",              # Grouping variable (e.g., dataset source)
    "OUTLET_COL": "Outlet_type",        # Categorical predictor (e.g., media type)
    "WORDLEN_COL": "Word_length",       # Continuous control (will be log-transformed)

    # DEPENDENT VARIABLES (count outcomes)
    "DEPENDENTS": ["Share_counts_Twitter", "Share_counts_Facebook"],

    # INDEPENDENT VARIABLES (discourse features)
    "NEWS_VALUES": [
        "Eliteness", "Personalisation", "Superlativeness", "Negativity",
        "Timeliness", "Unexpectedness", "Impact", "Space", "Positivity"
    ],

    # OPTIONAL COLUMN RENAMING
    # Use this if your column names differ from those above
    "RENAME": {
        # Example: "Share_counts Twitter": "Share_counts_Twitter",
        # Example: "Proximity": "Space",
    },

    # DATA PREPROCESSING
    "WINSOR_TOP": 0.99,        # Cap extreme values at 99th percentile (None to disable)
    "MIN_VARIANCE": 1e-6,      # Minimum variance for predictors (avoid constants)

    # DIAGNOSTIC THRESHOLDS
    "ZERO_THRESHOLD": 0.3,     # Flag if >30% zeros (suggests zero-inflation)
    "VIF_THRESHOLD": 10,       # Variance Inflation Factor threshold for multicollinearity
    "ALPHA_LEVEL": 0.05,       # Significance level for tests
}

In [4]:
print("\n" + "="*70)
print("DATA PREPARATION")
print("="*70)

if CONFIG["RENAME"]:
    df = df.rename(columns=CONFIG["RENAME"])
    print(f"Renamed {len(CONFIG['RENAME'])} columns")

CASE_COL = CONFIG["CASE_COL"]
OUTLET_COL = CONFIG["OUTLET_COL"]
WORDLEN_COL = CONFIG["WORDLEN_COL"]
NEWS_VALUES = CONFIG["NEWS_VALUES"]

DEPENDENTS = [c for c in CONFIG["DEPENDENTS"] if c in df.columns]
if not DEPENDENTS:
    raise ValueError("No dependent variables found. Check CONFIG['DEPENDENTS'] matches your data.")
print(f"Found {len(DEPENDENTS)} dependent variable(s): {DEPENDENTS}")

if CASE_COL not in df.columns:
    df[CASE_COL] = "All_articles"
    print(f"Created '{CASE_COL}' column (single dataset)")

for cat_col in [OUTLET_COL, CASE_COL]:
    if cat_col in df.columns:
        df[cat_col] = df[cat_col].astype("category")

for nv in NEWS_VALUES:
    if nv not in df.columns:
        df[nv] = 0
        print(f"Column '{nv}' not found - created as zeros")

df[NEWS_VALUES] = df[NEWS_VALUES].apply(pd.to_numeric, errors="coerce").fillna(0)

if WORDLEN_COL in df.columns:
    df[WORDLEN_COL] = pd.to_numeric(df[WORDLEN_COL], errors="coerce")
    df[WORDLEN_COL] = df[WORDLEN_COL].fillna(df[WORDLEN_COL].median())



DATA PREPARATION
Found 2 dependent variable(s): ['Share_counts_Twitter', 'Share_counts_Facebook']
Column 'Space' not found - created as zeros


In [5]:
print("\n Standardizing predictors...")
NV_STD = []
for nv in NEWS_VALUES:
    zcol = f"{nv}_std"
    col = df[nv].astype(float)

    if col.std(ddof=1) < CONFIG["MIN_VARIANCE"]:
        print(f"'{nv}' has zero/near-zero variance - excluding from analysis")
        continue

    df[zcol] = (col - col.mean()) / col.std(ddof=1)
    NV_STD.append(zcol)

print(f"Standardized {len(NV_STD)} predictors")


 Standardizing predictors...
'Space' has zero/near-zero variance - excluding from analysis
Standardized 8 predictors


In [6]:
def winsorize_top(series: pd.Series, p: float):
    if p is None:
        return series
    cap = series.quantile(p)
    n_capped = (series > cap).sum()
    if n_capped > 0:
        print(f"  → Capped {n_capped} values at {cap:.0f}")
    return series.clip(upper=cap)

WINSOR_TOP = CONFIG["WINSOR_TOP"]
MODELED_DEPS = []

if WINSOR_TOP:
    print(f"Winsorizing at {WINSOR_TOP*100}% percentile:")

for dep in DEPENDENTS:
    if dep not in df.columns:
        continue

    dep_w = f"{dep}_w" if WINSOR_TOP else dep

    if WINSOR_TOP:
        print(f"  {dep}:")
        df[dep_w] = winsorize_top(pd.to_numeric(df[dep], errors="coerce"), WINSOR_TOP)

    MODELED_DEPS.append(dep_w)

Winsorizing at 99.0% percentile:
  Share_counts_Twitter:
  → Capped 2 values at 37470
  Share_counts_Facebook:
  → Capped 2 values at 44771


In [7]:
def check_zero_inflation(series: pd.Series) -> dict:
    n_zeros = (series == 0).sum()
    pct_zeros = n_zeros / len(series)

    mean_val = series.mean()
    expected_zeros = np.exp(-mean_val) if mean_val > 0 else 1.0

    return {
        "n_zeros": n_zeros,
        "pct_zeros": pct_zeros,
        "expected_zeros": expected_zeros,
        "excess_zeros": pct_zeros > expected_zeros * 1.5,
        "high_zeros": pct_zeros > CONFIG["ZERO_THRESHOLD"]
    }

def calculate_vif(df_subset: pd.DataFrame) -> pd.DataFrame:
    """Calculate Variance Inflation Factor for multicollinearity detection"""
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    vif_data = pd.DataFrame()
    vif_data["Variable"] = df_subset.columns
    vif_data["VIF"] = [variance_inflation_factor(df_subset.values, i)
                       for i in range(len(df_subset.columns))]
    return vif_data.sort_values("VIF", ascending=False)

def compute_dispersion(model, dep_var):
    y = model.model.endog
    mu = model.fittedvalues
    residuals = (y - mu) ** 2
    dispersion = residuals.sum() / (len(y) - len(model.params))
    return dispersion

In [8]:
def build_formula(dep: str) -> str:
    rhs = []

    if WORDLEN_COL in df.columns:
        df[f"log_{WORDLEN_COL}"] = np.log1p(df[WORDLEN_COL].clip(lower=0))
        rhs.append(f"log_{WORDLEN_COL}")

    if OUTLET_COL in df.columns and df[OUTLET_COL].nunique() > 1:
        rhs.append(f"C({OUTLET_COL})")

    if CASE_COL in df.columns and df[CASE_COL].nunique() > 1:
        rhs.append(f"C({CASE_COL})")

    rhs += NV_STD

    return f"{dep} ~ " + " + ".join(rhs)

In [9]:
def fit_poisson(dep: str, verbose=True):
    fml = build_formula(dep)
    model = smf.glm(formula=fml, data=df, family=sm.families.Poisson()).fit(cov_type="HC3")

    if verbose:
        dispersion = compute_dispersion(model, dep)
        print(f"  Poisson dispersion: {dispersion:.2f} {'(overdispersed)' if dispersion > 2 else ''}")

    return model

def fit_negative_binomial(dep: str, verbose=True):
    fml = build_formula(dep)
    model = smf.glm(formula=fml, data=df, family=sm.families.NegativeBinomial()).fit(cov_type="HC3")

    if verbose:
        alpha = model.scale
        print(f"  NB alpha (dispersion): {alpha:.4f}")

    return model

def fit_zinb(dep: str, verbose=True):
    try:
        from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
        fml = build_formula(dep)

        print(" ZINB fitting requires manual configuration - using NB instead")
        return fit_negative_binomial(dep, verbose=False)
    except Exception as e:
        print(f" ZINB not available: {e}")
        return fit_negative_binomial(dep, verbose=False)

def model_to_irr(model, dep: str) -> pd.DataFrame:
    params = model.params
    conf = model.conf_int()

    out = pd.DataFrame({
        "term": params.index,
        "coef": params.values,
        "IRR": np.exp(params.values),
        "CI_lower": np.exp(conf[0].values),
        "CI_upper": np.exp(conf[1].values),
        "p_value": model.pvalues.values,
        "sig": ["***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""
                for p in model.pvalues.values]
    })

    out.insert(0, "dependent", dep)
    return out[["dependent", "term", "IRR", "CI_lower", "CI_upper", "coef", "p_value", "sig"]]

In [10]:
print("\n" + "="*70)
print("RUNNING ANALYSIS")
print("="*70)

all_results = []
diagnostics_report = []

for dep in MODELED_DEPS:
    print(f"\n{'='*70}")
    print(f"ANALYZING: {dep}")
    print(f"{'='*70}")

    df_clean = df[[dep] + NV_STD + [WORDLEN_COL, OUTLET_COL, CASE_COL]].dropna()
    dep_series = df_clean[dep]

    print(f"  Descriptive Statistics:")
    print(f"  N = {len(dep_series)}")
    print(f"  Mean = {dep_series.mean():.2f}")
    print(f"  Median = {dep_series.median():.2f}")
    print(f"  SD = {dep_series.std():.2f}")
    print(f"  Min = {dep_series.min():.0f}, Max = {dep_series.max():.0f}")

    zero_info = check_zero_inflation(dep_series)
    print(f"\n🔍 Zero-Inflation Check:")
    print(f"  Zeros: {zero_info['n_zeros']} ({zero_info['pct_zeros']:.1%})")
    print(f"  Expected (Poisson): {zero_info['expected_zeros']:.1%}")

    if zero_info['high_zeros']:
        print(f"HIGH zero proportion detected (>{CONFIG['ZERO_THRESHOLD']*100}%)")

    mode = CONFIG["MODE"]
    print(f" Fitting models (MODE: {mode})...")

    if mode == "SIMPLE":
        model = fit_negative_binomial(dep)
        model_type = "Negative Binomial"

    elif mode == "AUTO":
        poisson_model = fit_poisson(dep, verbose=True)
        nb_model = fit_negative_binomial(dep, verbose=True)

        if nb_model.aic < poisson_model.aic:
            model = nb_model
            model_type = "Negative Binomial"
            print(f"  ✓ Selected NB (AIC: {nb_model.aic:.1f} vs Poisson: {poisson_model.aic:.1f})")
        else:
            model = poisson_model
            model_type = "Poisson"
            print(f" Selected Poisson (AIC: {poisson_model.aic:.1f})")

    else:  # ADVANCED
        model = fit_zinb(dep)
        model_type = "ZINB/NB"


    irr_table = model_to_irr(model, dep)
    irr_table["model_type"] = model_type
    all_results.append(irr_table)


    diagnostics_report.append({
        "dependent": dep,
        "model": model_type,
        "n": len(dep_series),
        "zeros_pct": zero_info['pct_zeros'],
        "AIC": model.aic,
        "BIC": model.bic,
        "pseudo_R2": 1 - (model.deviance / model.null_deviance)
    })

    print(f"  Model fit complete: {model_type}")
    print(f"  AIC: {model.aic:.1f}")
    print(f"  Pseudo R²: {(1 - model.deviance/model.null_deviance):.3f}")


RUNNING ANALYSIS

ANALYZING: Share_counts_Twitter_w
  Descriptive Statistics:
  N = 192
  Mean = 2495.73
  Median = 7.00
  SD = 7960.09
  Min = 0, Max = 37470

🔍 Zero-Inflation Check:
  Zeros: 11 (5.7%)
  Expected (Poisson): 0.0%
 Fitting models (MODE: AUTO)...
  Poisson dispersion: 1193158.93 (overdispersed)
  NB alpha (dispersion): 1.0000
  ✓ Selected NB (AIC: 1794.2 vs Poisson: 18934.3)
  Model fit complete: Negative Binomial
  AIC: 1794.2
  Pseudo R²: 0.841

ANALYZING: Share_counts_Facebook_w
  Descriptive Statistics:
  N = 192
  Mean = 2666.95
  Median = 6.00
  SD = 8581.50
  Min = 0, Max = 44771

🔍 Zero-Inflation Check:
  Zeros: 26 (13.5%)
  Expected (Poisson): 0.0%
 Fitting models (MODE: AUTO)...
  Poisson dispersion: 1636562.44 (overdispersed)
  NB alpha (dispersion): 1.0000
  ✓ Selected NB (AIC: 1603.7 vs Poisson: 14905.9)
  Model fit complete: Negative Binomial
  AIC: 1603.7
  Pseudo R²: 0.869


In [11]:
print("\n" + "="*70)
print("COMPILING RESULTS")
print("="*70)

irr_results = pd.concat(all_results, ignore_index=True)

diagnostics_df = pd.DataFrame(diagnostics_report)

outfile = "nb_regression_results.xlsx"
with pd.ExcelWriter(outfile, engine="openpyxl") as writer:
    irr_results.to_excel(writer, sheet_name="IRR_Results", index=False)
    diagnostics_df.to_excel(writer, sheet_name="Model_Diagnostics", index=False)

    guide = pd.DataFrame({
        "Interpretation": [
            "IRR > 1: Positive association (predictor increases outcome)",
            "IRR < 1: Negative association (predictor decreases outcome)",
            "IRR = 1: No association",
            "Example: IRR=1.5 means 50% increase in outcome per 1-SD increase in predictor",
            "Significance: * p<0.05, ** p<0.01, *** p<0.001",
            "",
            "For categorical variables (e.g., Outlet_type):",
            "IRR compares to reference category",
            "",
            "Model Quality:",
            "Pseudo R² > 0.1 = weak, > 0.3 = moderate, > 0.5 = strong",
            "Lower AIC/BIC = better model fit"
        ]
    })
    guide.to_excel(writer, sheet_name="Interpretation_Guide", index=False)

print(f" Results exported to: {outfile}")

print("\n" + "="*70)
print("KEY FINDINGS (Significant Effects Only)")
print("="*70)

sig_results = irr_results[irr_results['p_value'] < 0.05].copy()
sig_results = sig_results[~sig_results['term'].str.contains("Intercept")]

if len(sig_results) > 0:
    display_cols = ["dependent", "term", "IRR", "CI_lower", "CI_upper", "p_value"]
    print(sig_results[display_cols].to_string(index=False))
else:
    print("No significant effects found at p < 0.05")

print("\n" + "="*70)
print("MODEL DIAGNOSTICS SUMMARY")
print("="*70)
print(diagnostics_df.to_string(index=False))

print(" Downloading results file...")
files.download(outfile)

print("\n" + "="*70)
print("ANALYSIS COMPLETE")
print("="*70)
print("\nNext steps:")
print("1. Review the downloaded Excel file for full results")
print("2. Check 'Model_Diagnostics' sheet for model quality")
print("3. Interpret IRRs using the 'Interpretation_Guide' sheet")
print("4. Report significant findings (p < 0.05) with confidence intervals")
print("\n For publication, report:")
print("   - Model type used (Poisson/NB/ZINB)")
print("   - Sample size and descriptives")
print("   - IRRs with 95% CIs")
print("   - Model fit statistics (AIC, Pseudo R²)")


COMPILING RESULTS
 Results exported to: nb_regression_results.xlsx

KEY FINDINGS (Significant Effects Only)
              dependent                      term         IRR    CI_lower     CI_upper       p_value
 Share_counts_Twitter_w C(Outlet_type)[T.Quality]    1.914196    1.110494     3.299564  1.942764e-02
 Share_counts_Twitter_w        C(Dataset)[T.Chad]    2.762474    1.476870     5.167187  1.470725e-03
 Share_counts_Twitter_w         C(Dataset)[T.USA] 3178.650633 1306.909792  7731.076702  9.535718e-71
 Share_counts_Twitter_w            Positivity_std    2.007247    1.226030     3.286250  5.603214e-03
Share_counts_Facebook_w C(Outlet_type)[T.Popular]    1.803899    1.147580     2.835578  1.057383e-02
Share_counts_Facebook_w C(Outlet_type)[T.Quality]    2.865140    1.868804     4.392663  1.379130e-06
Share_counts_Facebook_w        C(Dataset)[T.Chad]    3.253784    1.562076     6.777590  1.625695e-03
Share_counts_Facebook_w        C(Dataset)[T.Peru]    4.036562    1.206220    13.508

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


ANALYSIS COMPLETE

Next steps:
1. Review the downloaded Excel file for full results
2. Check 'Model_Diagnostics' sheet for model quality
3. Interpret IRRs using the 'Interpretation_Guide' sheet
4. Report significant findings (p < 0.05) with confidence intervals

 For publication, report:
   - Model type used (Poisson/NB/ZINB)
   - Sample size and descriptives
   - IRRs with 95% CIs
   - Model fit statistics (AIC, Pseudo R²)
